In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 27.1 MB/s eta 0:00:00


In [3]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=b876962c367d7add21da1ff272f3f092fb1c7799435d12206fbec71b80bcc582
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

from numpy.random import seed
seed(1337)
from tensorflow import random
random.set_seed(42)

import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics import classification_report, accuracy_score

import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation, Dense, Input, Conv1D, MaxPooling1D, Embedding, GlobalMaxPool1D, GlobalMaxPooling1D, SpatialDropout1D, LSTM, Flatten, BatchNormalization, Dropout
import transformers
import emoji
import re

In [5]:
data = pd.read_csv('/content/drive/MyDrive/capstone/balanced_lemmatized_cleaned_stocktwits.csv', index_col=0)

In [5]:
data.head()

,created_at,body,sentiment,raw_content
14,2020-12-15T14:38:18Z,"['going', 'right', 'support', 'even', 'superio...",0,$MSFT Going right through 214 support as if it...
49,2020-12-15T14:23:04Z,"['nobody', 'gonna', 'buy', 'expensive', 'as', ...",0,$AAPL nobody gonna buy expensive ass iPhones w...
61,2020-12-15T14:12:10Z,"['robinhood', 'peep', 'gonna', 'severely', 'di...",0,$AAPL Robinhood peeps gonna be severely disapp...
103,2020-12-15T13:33:52Z,"['always', 'dump', 'dump', 'dump']",0,$AAPL always dump dump dump.
106,2020-12-15T13:30:10Z,"['turd', 'going', 'anywhere', 'pathetic']",0,$AAPL why is this turd not going anywhere. Thi...


In [6]:
def preprocess(texts):
  # lowercase
  texts = texts.lower() # RoBERTa tokenizer is uncased
  # remove URLs
  texts = re.sub(r'https?://\S+', "", texts)
  texts = re.sub(r'www.\S+', "", texts)
  # remove '
  texts = texts.replace('&#39;', "'")
  # remove symbol names
  texts = re.sub(r'(\#)(\S+)', r'hashtag_\2', texts)
  texts = re.sub(r'(\$)([A-Za-z]+)', r'cashtag_\2', texts)
  # remove usernames
  texts = re.sub(r'(\@)(\S+)', r'mention_\2', texts)
  # demojize
  texts = emoji.demojize(texts, delimiters=("", " "))

  return texts.strip()

In [7]:
data['body'] = data['raw_content']
data['body'] = data.body.apply(preprocess)
data.head()

,created_at,body,sentiment,raw_content
14,2020-12-15T14:38:18Z,cashtag_msft going right through 214 support a...,0,$MSFT Going right through 214 support as if it...
49,2020-12-15T14:23:04Z,cashtag_aapl nobody gonna buy expensive ass ip...,0,$AAPL nobody gonna buy expensive ass iPhones w...
61,2020-12-15T14:12:10Z,cashtag_aapl robinhood peeps gonna be severely...,0,$AAPL Robinhood peeps gonna be severely disapp...
103,2020-12-15T13:33:52Z,cashtag_aapl always dump dump dump.,0,$AAPL always dump dump dump.
106,2020-12-15T13:30:10Z,cashtag_aapl why is this turd not going anywhe...,0,$AAPL why is this turd not going anywhere. Thi...


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['body'], data['sentiment'], test_size=0.2, random_state=42)

In [8]:
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train: (690056,)
X_test: (172514,)
y_train: (690056,)
y_test: (172514,)


In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-cased")
# tokenizer = transformers.AutoTokenizer.from_pretrained('ProsusAI/finbert')
# Maximum sequence size for BERT is 512
def regular_encode(texts, tokenizer, maxlen):
    enc_di = tokenizer.batch_encode_plus(texts, return_token_type_ids = False, padding = 'max_length', max_length = maxlen, truncation=True)
    return np.array(enc_di['input_ids'])

In [10]:
Xtrain_encoded = regular_encode(X_train.astype('str'), tokenizer, maxlen = 31)
Xtest_encoded = regular_encode(X_test.astype('str'),  tokenizer, maxlen = 31)
# Xtrain_encoded = regular_encode(X_train.tolist(), tokenizer, maxlen = 31)
# Xtest_encoded = regular_encode(X_test.tolist(),  tokenizer, maxlen = 31)

In [11]:
max_len = 31
input_word_ids = tf.keras.layers.Input(shape = (max_len,), dtype = tf.int32, name = "input_word_ids")
transformer_layer = transformers.TFAutoModel.from_pretrained('bert-base-uncased')
# transformer_layer =  transformers.TFAutoModel.from_pretrained('ProsusAI/finbert')
sequence_output = transformer_layer(input_word_ids)[0]

cls_token = sequence_output[:, 0, :]
x = tf.keras.layers.Dense(250, activation = 'relu')(cls_token)
x = tf.keras.layers.Dropout(0.5)(x)
out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.Model(inputs = input_word_ids, outputs = out)
model.compile(tf.keras.optimizers.Adam(learning_rate = 5e-6), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 31)]             0         
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  109482240
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             31, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                             

In [12]:
#creating the training and testing dataset.
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE 
train_dataset = (tf.data.Dataset.from_tensor_slices((Xtrain_encoded, y_train)).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))
test_dataset = (tf.data.Dataset.from_tensor_slices((Xtest_encoded, y_test)).shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))

In [13]:
train_history_bert = model.fit(train_dataset, validation_data=test_dataset, epochs = 3)

Epoch 1/3


21565/21565 [==============================] - 5330s 245ms/step - loss: 0.5340 - accuracy: 0.7261 - val_loss: 0.4707 - val_accuracy: 0.7701
Epoch 2/3
21565/21565 [==============================] - 5242s 243ms/step - loss: 0.4596 - accuracy: 0.7787 - val_loss: 0.4417 - val_accuracy: 0.7884
Epoch 3/3
21565/21565 [==============================] - 5236s 243ms/step - loss: 0.4240 - accuracy: 0.7996 - val_loss: 0.4304 - val_accuracy: 0.7987


In [28]:
train_history_finbert = model.fit(train_dataset, validation_data=test_dataset, epochs = 3)

Epoch 1/3


21565/21565 [==============================] - 5227s 240ms/step - loss: 0.4457 - accuracy: 0.7861 - val_loss: 0.3843 - val_accuracy: 0.8235
Epoch 2/3
21565/21565 [==============================] - 5135s 238ms/step - loss: 0.3639 - accuracy: 0.8349 - val_loss: 0.3604 - val_accuracy: 0.8376
Epoch 3/3
21565/21565 [==============================] - 5134s 238ms/step - loss: 0.3227 - accuracy: 0.8569 - val_loss: 0.3592 - val_accuracy: 0.8415


In [ ]:
model.save('capstone_BERT.h5')